In [35]:
import json
import pandas as pd
import numpy as np
import re
from sqlalchemy import create_engine
import psycopg2
import time

In [48]:
# 1. Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):

    movie_1 = dict(movie)
    alternate_titles = {}

    for key in ['Also known as','Arabic','Cantonese','Chinese','French',
                'Hangul','Hebrew','Hepburn','Japanese','Literally',
                'Mandarin','McCune-Reischauer','Original title','Polish',
                'Revised Romanization','Romanized','Russian',
                'Simplified','Traditional','Yiddish']:
        if key in movie:
            alternate_titles[key] = movie[key]
            movie.pop(key)
    if len(alternate_titles) > 0:
        movie['alternate_titles'] = alternate_titles

    # merge column names
    def change_name_column(old_name, new_name):
        if old_name in movie:
            movie[new_name] = movie.pop(old_name)
    change_name_column('Adaptation by', 'Writer')
    change_name_column('Country of origin', 'Country')
    change_name_column('Directed by', 'Director')
    change_name_column('Distributed by', 'Distributor')
    change_name_column('Edited by', 'Editor')
    change_name_column('Editor(s)', 'Editor')    
    change_name_column('Length', 'Running time')
    change_name_column('Original release', 'Release Date')
    change_name_column('Music by', 'Composer')
    change_name_column('Composer(s)', 'Composer')
    change_name_column('Produced by', 'Producer')
    change_name_column('Producer', 'Producer')
    change_name_column('Producer(s)', 'Producer')
    change_name_column('Productioncompanies ', 'Production company')
    change_name_column('Productioncompany ', 'Production company')
    change_name_column('Production company(s)', 'Production company')
    change_name_column('Original network', 'Original Network')
    change_name_column('Released', 'Release Date')
    change_name_column('Screen story by', 'Writer')
    change_name_column('Screenplay by', 'Writer')
    change_name_column('Story by', 'Writer')
    change_name_column('Theme music composer', 'Composer')
    change_name_column('Written by', 'Writer')
    change_name_column('year', 'Year')
    change_name_column('Original language(s)', 'Original Language')

    return movie

In [45]:
# Wikipedia data
wiki_file = 'C:/Users/sophi/OneDrive/Desktop/GitHubRepo/ETL_Movies/wikipedia_movies.json'
# Kaggle metadata
kaggle_file = 'C:/Users/sophi/OneDrive/Desktop/GitHubRepo/ETL_Movies/movies_metadata.csv'
# MovieLens rating data.
ratings_file = 'C:/Users/sophi/OneDrive/Desktop/GitHubRepo/ETL_Movies/ratings.csv'

In [46]:
def e_t_l():
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.
    kaggle_metadata = pd.read_csv(kaggle_file, low_memory=False)
    ratings = pd.read_csv(ratings_file)
    
    # 3. Open the read the Wikipedia data JSON file.
    with open (wiki_file) as file: 
        wiki_movies_raw = json.load(file)
    
    # 3. Write a list comprehension to filter out TV shows.
    wiki_movies = [movie for movie in wiki_movies_raw 
               if ('Director' in movie or 'Directed by' in movie) 
               and 'imdb_link' in movie and 'No. of episodes' not in movie]
   

    # 4. Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    cleaned_movies = [clean_movie(movie) for movie in wiki_movies]


    # 5. Read in the cleaned movies list from Step 4 as a DataFrame.
    wiki_movies_df = pd.DataFrame(cleaned_movies)


    # 6. Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    try:
        wiki_movies_df['imdb_id'] = wiki_movies_df['imdb_link'].str.extract(r'(tt\d{7})')
        wiki_movies_df.drop_duplicates(subset='imdb_id', inplace=True)
        
    except Error: 
        print("error")

    #  7. Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    
    wiki_columns_keep = [column for column in wiki_movies_df.columns 
                            if wiki_movies_df[column].isnull().sum() < len(wiki_movies_df) * 0.9]

    # 8. Create a variable that will hold the non-null values from the “Box office” column.
    box_office_data = wiki_movies_df['Box office'].dropna()
    
    # 9. Convert the box office data created in Step 8 to string values using the lambda and join functions.
    box_office_data[box_office_data.map(lambda x: type(x) != str)]
    box_office_data = box_office_data.apply(lambda x: ' '.join(x) if type(x) == list else x)

    # 10. Write a regular expression to match the six elements of "form_one" of the box office data.
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illion'
    box_office_data.str.contains(form_one, flags=re.IGNORECASE, na=False).sum()
    matches_form_one = box_office_data.str.contains(form_one, flags=re.IGNORECASE, na=False)

    # 11. Write a regular expression to match the three elements of "form_two" of the box office data.
    form_two = r'\$\d{1,3}(?:,\d{3})+'
    box_office_data.str.contains(form_two, flags=re.IGNORECASE, na=False).sum()
    matches_form_two = box_office_data.str.contains(form_two, flags=re.IGNORECASE, na=False)

    # 12. Add the parse_dollars function.
    def parse_dollars(s):

        if type(s) != str:
            return np.nan

        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

            s = re.sub('\$|\s|[a-zA-Z]','', s)

            value = float(s) * 10**6

            return value

        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

            s = re.sub('\$|\s|[a-zA-Z]','', s)

            value = float(s) * 10**9

            return value

        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

            s = re.sub('\$|,','', s)

            value = float(s)

            return value
        
        else:
            return np.nan

    
    # 13. Clean the box office column in the wiki_movies_df DataFrame.
    wiki_movies_df['box_office_data'] = box_office_data.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    
    # 14. Clean the budget column in the wiki_movies_df DataFrame.
    budget = wiki_movies_df['Budget'].dropna()
    budget = budget.map(lambda x: ' '.join(x) if type(x) == list else x)
    budget = budget.str.replace(r'\$.*[-—–](?![a-z])', '$', regex=True)
    matches_form_one = budget.str.contains(form_one, flags=re.IGNORECASE, na=False)
    matches_form_two = budget.str.contains(form_two, flags=re.IGNORECASE, na=False)
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)

    # 15. Clean the release date column in the wiki_movies_df DataFrame.
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    date_form_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]?\d,\s\d{4}'
    date_form_two = r'\d{4}.[01]\d.[0123]\d'
    date_form_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_form_four = r'\d{4}'
    release_date = release_date.str.extract(f'({date_form_one}|{date_form_two}|{date_form_three}|{date_form_four})', flags=re.IGNORECASE)

    # 16. Clean the running time column in the wiki_movies_df DataFrame.
    run_time = wiki_movies_df['Running time'].dropna().apply(lambda x: ' '.join(x) if type(x) == list else x)
    run_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False).sum()
    run_time[run_time.str.contains(r'^\d*\s*minutes$', flags=re.IGNORECASE, na=False) != True]
    run_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False).sum()
    run_time[run_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE, na=False) != True]
    run_time_extract = run_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    run_time_extract = run_time_extract.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    wiki_movies_df['run_time'] = run_time_extract.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
    
    # Return three variables. The first is the wiki_movies_df DataFrame
    
    return wiki_movies_df, kaggle_metadata, ratings 

In [49]:
# 18. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = e_t_l()

C:\Users\sophi\anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:100: FutureWarning: The default value of regex will change from True to False in a future version.


In [50]:
# 19. Set the wiki_movies_df equal to the wiki_file variable. 
wiki_movies_df = wiki_file

In [51]:
# 20. Check that the wiki_movies_df DataFrame looks like this. 
wiki_movies_df.head()

,url,imdb_link,title,Based on,Starring,Narrated by,Cinematography,Release date,Country,Language,...,Recorded,Venue,Label,Animation by,Color process,McCune–Reischauer,imdb_id,box_office_data,budget,run_time
0,https://en.wikipedia.org/wiki/The_Adventures_o...,https://www.imdb.com/title/tt0098987/,The Adventures of Ford Fairlane,"[Characters, by Rex Weiner]","[Andrew Dice Clay, Wayne Newton, Priscilla Pre...","Andrew ""Dice"" Clay",Oliver Wood,"[July 11, 1990, (, 1990-07-11, )]",United States,English,...,NaN,NaN,NaN,NaN,NaN,NaN,tt0098987,21400000.0,20000000.0,102.0
1,"https://en.wikipedia.org/wiki/After_Dark,_My_S...",https://www.imdb.com/title/tt0098994/,"After Dark, My Sweet","[the novel, After Dark, My Sweet, by, Jim Thom...","[Jason Patric, Rachel Ward, Bruce Dern, George...",NaN,Mark Plummer,"[May 17, 1990, (, 1990-05-17, ), (Cannes Film ...",United States,English,...,NaN,NaN,NaN,NaN,NaN,NaN,tt0098994,2700000.0,6000000.0,114.0
2,https://en.wikipedia.org/wiki/Air_America_(film),https://www.imdb.com/title/tt0099005/,Air America,"[Air America, by, Christopher Robbins]","[Mel Gibson, Robert Downey Jr., Nancy Travis, ...",NaN,Roger Deakins,"[August 10, 1990, (, 1990-08-10, )]",United States,"[English, Lao]",...,NaN,NaN,NaN,NaN,NaN,NaN,tt0099005,57718089.0,35000000.0,113.0
3,https://en.wikipedia.org/wiki/Alice_(1990_film),https://www.imdb.com/title/tt0099012/,Alice,NaN,"[Alec Baldwin, Blythe Danner, Judy Davis, Mia ...",NaN,Carlo Di Palma,"[December 25, 1990, (, 1990-12-25, )]",United States,English,...,NaN,NaN,NaN,NaN,NaN,NaN,tt0099012,7331647.0,12000000.0,106.0
4,https://en.wikipedia.org/wiki/Almost_an_Angel,https://www.imdb.com/title/tt0099018/,Almost an Angel,NaN,"[Paul Hogan, Elias Koteas, Linda Kozlowski]",NaN,Russell Boyd,"December 19, 1990",US,English,...,NaN,NaN,NaN,NaN,NaN,NaN,tt0099018,6939946.0,25000000.0,95.0


In [52]:
# 21. Check that wiki_movies_df DataFrame columns are correct. 
wiki_movies_df.columns.to_list()

['url',
 'imdb_link',
 'title',
 'Based on',
 'Starring',
 'Narrated by',
 'Cinematography',
 'Release date',
 'Country',
 'Language',
 'Budget',
 'Box office',
 'Director',
 'Distributor',
 'Editor',
 'Composer',
 'Producer',
 'Production company',
 'Writer',
 'Year',
 'Genre',
 'Release Date',
 'Original Network',
 'Original Language',
 'Executive producer(s)',
 'Production location(s)',
 'Picture format',
 'Audio format',
 'Voices of',
 'Followed by',
 'Created by',
 'Preceded by',
 'Suggested by',
 'alternate_titles',
 'Recorded',
 'Venue',
 'Label',
 'Animation by',
 'Color process',
 'McCune–Reischauer',
 'imdb_id',
 'box_office_data',
 'budget',
 'run_time']